In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Function Overview

- **Function Name:** `auto_label`
- **Parameters:**
  - `image_path` (str): The file path of the image for which the XML annotation will be generated.
  - `bndbox` (list of tuples): A list of bounding boxes, where each bounding box is represented as a tuple `(xmin, ymin, xmax, ymax)`.
  - `class_name` (str): The class name of the object(s) to be annotated. Defaults to `'rebar'`.

## Code Details

1. **Image Dimensions**: The script uses the `PIL` library to open the image and extract its dimensions.
2. **XML Structure**: An XML structure is created with the root element `<annotation>`, which includes child elements for filename, size (width, height, depth), and bounding boxes for each object.
3. **File Saving**: The generated XML is saved to a file with the same name as the image but with an `.xml` extension.

# Instructions to Change the XML Saving Path

By default, the XML file is saved in the same directory as the image file and has the same base name with an `.xml` extension. To change the XML saving path, follow these steps:

1. **Locate the XML Saving Code**:
   Find the following line in the `auto_label` function where the XML file path is determined:
   ```python
   xml_file_path = image_path.replace('.jpg', '.xml').replace('.png', '.xml')

In [ ]:
import xml.etree.ElementTree as ET

def auto_label(image_path, bndbox, class_name='rebar'):
    # Load image to get dimensions
    from PIL import Image
    with Image.open(image_path) as img:
        width, height = img.size

    # Create XML structure
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'filename').text = image_path.split('/')[-1]

    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str(width)
    ET.SubElement(size, 'height').text = str(height)
    ET.SubElement(size, 'depth').text = '3'  # Assuming RGB image

    for bbox in bndbox:
        obj = ET.SubElement(annotation, 'object')
        ET.SubElement(obj, 'name').text = class_name
        ET.SubElement(obj, 'pose').text = 'Unspecified'
        ET.SubElement(obj, 'truncated').text = '0'
        ET.SubElement(obj, 'difficult').text = '0'

        bndbox_element = ET.SubElement(obj, 'bndbox')
        ET.SubElement(bndbox_element, 'xmin').text = str(bbox[0])
        ET.SubElement(bndbox_element, 'ymin').text = str(bbox[1])
        ET.SubElement(bndbox_element, 'xmax').text = str(bbox[2])
        ET.SubElement(bndbox_element, 'ymax').text = str(bbox[3])

    # Convert the XML structure to a string
    xml_str = ET.tostring(annotation, encoding='utf-8', method='xml').decode()

    # Write XML string to a file
    xml_file_path = image_path.replace('.jpg', '.xml').replace('.png', '.xml')
    with open(xml_file_path, 'w') as f:
        f.write(xml_str)

    print(f'XML file saved to {xml_file_path}')


In [ ]:
import cv2
import numpy as np
from PIL import Image
import math
import tensorflow as tf
model_path = 'rebar_preTrained_model.tflite'
classes = ['rebar']


def preprocess_image(image_path, input_size):
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image

def detect_objects(interpreter, image, threshold):
  signature_fn = interpreter.get_signature_runner()
  output = signature_fn(images=image)
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])
  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results

def run_odt_and_draw_results(image_path, interpreter, threshold=0.25, iou_threshold=0.2):
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)
  boxes = np.array([result['bounding_box'] for result in results])
  scores = np.array([result['score'] for result in results])
  selected_indices = tf.image.non_max_suppression(
      boxes, scores, max_output_size=400, iou_threshold=iou_threshold
  ).numpy()
  selected_results = [results[i] for i in selected_indices]
  print(selected_results)
  original_image_np = original_image.numpy().astype(np.uint8)
  bndbox = []
  for obj in selected_results:
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])
    bndbox.append([xmin,ymin,xmax,ymax])
  auto_label(image_path,bndbox)


# Instructions for Processing Multiple Images in a Folder

To process every image in a folder, follow these steps:

1. **Import Required Libraries**

   Make sure to import the necessary libraries at the beginning of your script:
   - `os` for interacting with the file system.

2. **Define the Directory and Model Path**

   Set the path to the folder containing your images .


3. **Iterate Through the Images in the Directory**

   Use the `os.listdir` function to list all files in the specified directory. For each image file (e.g., `.png`, `.jpg`, `.jpeg`), do the following:
   - Construct the full file path.
   - Call the `run_odt_and_draw_results` function with the image path and interpreter.

4. **Ensure Proper File Handling**

   - Make sure that the directory where you save processed images exists or create it if necessary.
   - Handle any potential errors, such as file not found or permission issues, gracefully.

By following these steps, you will be able to process each image in the specified folder

In [ ]:
from tensorflow.keras.utils import array_to_img
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

detection_result_image = run_odt_and_draw_results(
    "/content/drive/MyDrive/IMG_57812.png",
    interpreter
)

[{'bounding_box': array([0.5985279 , 0.7409323 , 0.64430726, 0.7814608 ], dtype=float32), 'class_id': 0.0, 'score': 0.70703125}, {'bounding_box': array([0.5377199 , 0.6485695 , 0.58349925, 0.68801564], dtype=float32), 'class_id': 0.0, 'score': 0.67578125}, {'bounding_box': array([0.1538323, 0.6178968, 0.2055427, 0.6578804], dtype=float32), 'class_id': 0.0, 'score': 0.6640625}, {'bounding_box': array([0.58321065, 0.6674251 , 0.6283745 , 0.70379406], dtype=float32), 'class_id': 0.0, 'score': 0.65234375}, {'bounding_box': array([0.5962976 , 0.7001193 , 0.6414615 , 0.73698395], dtype=float32), 'class_id': 0.0, 'score': 0.65234375}, {'bounding_box': array([0.54393613, 0.60755557, 0.5916128 , 0.6454318 ], dtype=float32), 'class_id': 0.0, 'score': 0.640625}, {'bounding_box': array([0.18592831, 0.65137273, 0.23763871, 0.6902885 ], dtype=float32), 'class_id': 0.0, 'score': 0.62890625}, {'bounding_box': array([0.14092824, 0.45104742, 0.19058082, 0.4894399 ], dtype=float32), 'class_id': 0.0, 'sco